In [ ]:
# Setup sys.path for CA12 package imports
import sys
import os
sys.path.insert(0, os.path.abspath("."))
sys.path.insert(0, os.path.abspath(".."))

# Import all required modules
from utils.setup import (
    device, n_gpus, agent_colors, performance_colors, ma_config, policy_config,
    MultiAgentConfig, PolicyConfig
)
from agents.cooperative_learning import (
    Actor, Critic, MADDPGAgent, MADDPG, ReplayBuffer, VDNAgent, VDN
)
from agents.advanced_policy import PPONetwork, PPOAgent, SACAgent, GAEBuffer
from agents.distributed_rl import (
    ParameterServer, A3CWorker, IMPALALearner, DistributedPPOCoordinator, 
    EvolutionaryStrategy, demonstrate_parameter_server, demonstrate_evolutionary_strategy
)
from agents.meta_learning import (
    MAMLAgent, OpponentModel, PopulationBasedTraining, SelfPlayTraining,
    demonstrate_maml, demonstrate_opponent_modeling, demonstrate_population_training
)
from experiments.game_theory import (
    GameTheoryUtils, MultiAgentEnvironment, demonstrate_game_theory, test_multi_agent_env
)
from experiments.communication import (
    CommunicationChannel, AttentionCommunication, CoordinationMechanism,
    MarketBasedCoordination, HierarchicalCoordination, EmergentCommunicationAgent,
    demonstrate_communication, demonstrate_coordination, demonstrate_emergent_communication
)
from experiments.applications import (
    ResourceAllocationEnvironment, AutonomousVehicleEnvironment, SmartGridEnvironment,
    MultiAgentGameTheoryAnalyzer, demonstrate_resource_allocation, 
    demonstrate_autonomous_vehicles, demonstrate_smart_grid
)
from experiments.training_framework import MultiAgentTrainingOrchestrator

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Configured sys.path for CA12 imports")
print("✅ All CA12 modules imported successfully!")


Configured sys.path for CA12 imports


# Table of Contents

1. [Abstract](#abstract)
2. [1. Introduction](#1-introduction)
   - [1.1 Motivation](#11-motivation)
   - [1.2 Learning Objectives](#12-learning-objectives)
   - [1.3 Prerequisites](#13-prerequisites)
   - [1.4 Course Information](#14-course-information)
3. [2. Multi-Agent RL Foundations](#2-multi-agent-rl-foundations)
   - [2.1 Game Theory Basics](#21-game-theory-basics)
   - [2.2 Cooperative vs Competitive Settings](#22-cooperative-vs-competitive-settings)
   - [2.3 Non-Stationarity and Partial Observability](#23-non-stationarity-and-partial-observability)
   - [2.4 Centralized Training Decentralized Execution](#24-centralized-training-decentralized-execution)
4. [3. Cooperative Multi-Agent Learning](#3-cooperative-multi-agent-learning)
   - [3.1 Multi-Agent Actor-Critic (MAAC)](#31-multi-agent-actor-critic-maac)
   - [3.2 Value Decomposition Networks (VDN)](#32-value-decomposition-networks-vdn)
   - [3.3 Counterfactual Multi-Agent Policy Gradients (COMA)](#33-counterfactual-multi-agent-policy-gradients-coma)
   - [3.4 Credit Assignment and Reward Shaping](#34-credit-assignment-and-reward-shaping)
5. [4. Advanced Policy Gradient Methods](#4-advanced-policy-gradient-methods)
   - [4.1 Proximal Policy Optimization (PPO) Variants](#41-proximal-policy-optimization-ppo-variants)
   - [4.2 Trust Region Policy Optimization (TRPO)](#42-trust-region-policy-optimization-trpo)
   - [4.3 Soft Actor-Critic (SAC) Extensions](#43-soft-actor-critic-sac-extensions)
   - [4.4 Generalized Advantage Estimation (GAE)](#44-generalized-advantage-estimation-gae)
6. [5. Competitive Multi-Agent Learning](#5-competitive-multi-agent-learning)
   - [5.1 Self-Play and Population-Based Training](#51-self-play-and-population-based-training)
   - [5.2 Multi-Agent Deep Deterministic Policy Gradient (MADDPG)](#52-multi-agent-deep-deterministic-policy-gradient-maddpg)
   - [5.3 Nash Equilibrium Learning](#53-nash-equilibrium-learning)
   - [5.4 Adversarial Training](#54-adversarial-training)
7. [6. Distributed Training and Scalability](#6-distributed-training-and-scalability)
   - [6.1 Distributed Policy Gradients](#61-distributed-policy-gradients)
   - [6.2 Asynchronous Multi-Agent Learning](#62-asynchronous-multi-agent-learning)
   - [6.3 Communication and Coordination](#63-communication-and-coordination)
   - [6.4 Scalability Challenges](#64-scalability-challenges)
8. [7. Implementation and Experimental Design](#7-implementation-and-experimental-design)
   - [7.1 Environment Setup](#71-environment-setup)
   - [7.2 Multi-Agent Environment Design](#72-multi-agent-environment-design)
   - [7.3 Training Procedures](#73-training-procedures)
   - [7.4 Evaluation Metrics](#74-evaluation-metrics)
9. [8. Results and Analysis](#8-results-and-analysis)
   - [8.1 Cooperative Learning Performance](#81-cooperative-learning-performance)
   - [8.2 Competitive Learning Analysis](#82-competitive-learning-analysis)
   - [8.3 Scalability Studies](#83-scalability-studies)
   - [8.4 Comparative Analysis](#84-comparative-analysis)
10. [9. Results and Discussion](#9-results-and-discussion)
    - [9.1 Summary of Findings](#91-summary-of-findings)
    - [9.2 Theoretical Contributions](#92-theoretical-contributions)
    - [9.3 Practical Implications](#93-practical-implications)
    - [9.4 Limitations and Future Work](#94-limitations-and-future-work)
    - [9.5 Conclusions](#95-conclusions)
11. [References](#references)
12. [Appendix A: Implementation Details](#appendix-a-implementation-details)
    - [A.1 Modular Architecture](#a1-modular-architecture)
    - [A.2 Code Quality Features](#a2-code-quality-features)
    - [A.3 Performance Considerations](#a3-performance-considerations)

---

# Computer Assignment 12: Multi-Agent Reinforcement Learning and Advanced Policy Methods

## Abstract

This assignment presents a comprehensive study of multi-agent reinforcement learning and advanced policy methods, exploring the challenges and solutions for learning in multi-agent environments with both cooperative and competitive settings. We implement and analyze various multi-agent algorithms including Multi-Agent Actor-Critic (MAAC), Value Decomposition Networks (VDN), Counterfactual Multi-Agent Policy Gradients (COMA), and advanced policy optimization methods. The assignment covers game theory foundations, centralized training decentralized execution (CTDE) paradigms, and distributed training approaches. Through systematic experimentation, we demonstrate the effectiveness of different multi-agent learning strategies and their applications to complex multi-agent scenarios.

**Keywords:** Multi-agent reinforcement learning, game theory, cooperative learning, competitive learning, MAAC, VDN, COMA, MADDPG, distributed training, policy optimization

## 1. Introduction

Multi-agent reinforcement learning represents a significant extension of single-agent reinforcement learning, where multiple agents interact in a shared environment and must learn to coordinate, compete, or coexist effectively [1]. Unlike single-agent settings, multi-agent environments introduce additional challenges such as non-stationarity, partial observability, and the need for coordination mechanisms. These challenges have led to the development of specialized algorithms and training paradigms that address the unique aspects of multi-agent learning.

### 1.1 Motivation

Multi-agent reinforcement learning addresses several important real-world scenarios:

- **Cooperative Systems**: Teams of agents working together to achieve common goals
- **Competitive Systems**: Agents competing for limited resources or conflicting objectives
- **Mixed Systems**: Environments with both cooperative and competitive elements
- **Scalability**: Learning in environments with large numbers of agents
- **Communication**: Enabling agents to share information and coordinate actions

### 1.2 Learning Objectives

By the end of this assignment, you will understand:

1. **Multi-Agent RL Foundations**:
   - Game theory basics (Nash equilibrium, Pareto optimality)
   - Cooperative vs competitive multi-agent settings
   - Non-stationarity and partial observability challenges
   - Centralized training decentralized execution (CTDE)

2. **Cooperative Multi-Agent Learning**:
   - Multi-Agent Actor-Critic (MAAC) methods
   - Value Decomposition Networks (VDN)
   - Counterfactual Multi-Agent Policy Gradients (COMA)
   - Credit assignment and reward shaping

3. **Advanced Policy Gradient Methods**:
   - Proximal Policy Optimization (PPO) variants
   - Trust Region Policy Optimization (TRPO)
   - Soft Actor-Critic (SAC) extensions
   - Generalized Advantage Estimation (GAE)

4. **Competitive Multi-Agent Learning**:
   - Self-play and population-based training
   - Multi-Agent Deep Deterministic Policy Gradient (MADDPG)
   - Nash equilibrium learning
   - Adversarial training approaches

### 1.3 Prerequisites

Before starting this assignment, ensure you have:

- **Mathematical Background**:
  - Game theory fundamentals
  - Probability theory and statistics
  - Linear algebra and optimization
  - Multi-agent systems theory

- **Technical Skills**:
  - Python programming and PyTorch
  - Deep learning and neural networks
  - Reinforcement learning fundamentals
  - Distributed computing basics

### 1.4 Course Information

- **Course**: Deep Reinforcement Learning
- **Session**: 12
- **Topic**: Multi-Agent Reinforcement Learning and Advanced Policy Methods
- **Focus**: Cooperative/competitive multi-agent systems, advanced policy optimization, and distributed training

4. **Distributed Reinforcement Learning**:
- Asynchronous Advantage Actor-Critic (A3C)
- IMPALA architecture and V-trace
- Parameter server architectures
- Evolutionary strategies for RL

5. **Communication and Coordination**:
- Emergent communication protocols
- Attention-based message passing
- Market-based coordination mechanisms
- Hierarchical coordination structures

6. **Meta-Learning in Multi-Agent Systems**:
- Model-Agnostic Meta-Learning (MAML) for MARL
- Few-shot adaptation and opponent modeling
- Population-based training and self-play
- Continual learning in dynamic environments

7. **Real-World Applications**:
- Autonomous vehicle coordination
- Smart grid management
- Robotics swarm coordination
- Financial trading systems

### Prerequisites

Before starting this notebook, ensure you have:

- **Mathematical Background**:
- Game theory fundamentals (equilibria, utilities)
- Advanced probability and stochastic processes
- Multi-variable optimization
- Information theory and communication

- **Programming Skills**:
- Advanced PyTorch (distributed training, multi-GPU)
- Parallel computing and asynchronous programming
- Network communication and message passing
- Large-scale system design and orchestration

- **Reinforcement Learning Knowledge**:
- Policy gradient methods (REINFORCE, Actor-Critic)
- Multi-agent MDP formulations
- Experience replay and stability techniques
- Continuous control and action spaces

- **Previous Course Knowledge**:
- CA1-CA6: Complete RL fundamentals and algorithms
- CA7-CA9: Advanced policy methods and actor-critic
- CA10-CA11: Model-based RL and world models
- Strong foundation in distributed computing

### Roadmap

This notebook follows a structured progression from multi-agent foundations to advanced applications:

1. **Section 1: Multi-Agent Foundations and Game Theory** (45 min)
- Game theory basics and equilibrium concepts
- Multi-agent MDP formulations
- Cooperation vs competition spectrum
- Mathematical foundations of MARL

2. **Section 2: Cooperative Multi-Agent Learning** (60 min)
- Centralized training decentralized execution
- Multi-Agent Actor-Critic (MAAC) methods
- Value decomposition approaches
- Credit assignment techniques

3. **Section 3: Advanced Policy Gradient Methods** (60 min)
- PPO, TRPO, and SAC variants
- Generalized Advantage Estimation
- Multi-agent policy gradient extensions
- Advanced advantage computation

4. **Section 4: Distributed Reinforcement Learning** (45 min)
- Asynchronous methods (A3C, IMPALA)
- Parameter server architectures
- Evolutionary strategies
- Scalability and fault tolerance

5. **Section 5: Communication and Coordination** (45 min)
- Emergent communication protocols
- Attention-based message passing
- Market-based coordination
- Hierarchical coordination structures

6. **Section 6: Meta-Learning and Adaptation** (45 min)
- MAML for multi-agent systems
- Few-shot adaptation and opponent modeling
- Population-based training
- Continual learning approaches

7. **Section 7: Comprehensive Applications** (60 min)
- Autonomous vehicle coordination
- Smart grid management
- Robotics swarm coordination
- Financial trading systems

### Project Structure

This notebook uses a modular implementation organized as follows:

```
CA12/
├── agents/                     # Multi-agent RL agents
│   ├── cooperative/           # Cooperative learning agents
│   │   ├── maddpg.py          # Multi-Agent DDPG
│   │   ├── vdqn.py            # Value Decomposition Networks
│   │   ├── coma.py            # Counterfactual Multi-Agent Policy Gradients
│   │   └── maac.py            # Multi-Agent Actor-Critic
│   ├── competitive/           # Competitive learning agents
│   │   ├── self_play.py       # Self-play training
│   │   ├── population_based.py # Population-based methods
│   │   └── adversarial.py     # Adversarial training
│   ├── advanced_policy/       # Advanced policy methods
│   │   ├── ppo.py             # Proximal Policy Optimization
│   │   ├── trpo.py            # Trust Region Policy Optimization
│   │   ├── sac.py             # Soft Actor-Critic
│   │   └── gae.py             # Generalized Advantage Estimation
│   ├── distributed/           # Distributed RL agents
│   │   ├── a3c.py             # Asynchronous Advantage Actor-Critic
│   │   ├── impala.py          # IMPALA architecture
│   │   ├── parameter_server.py # Parameter server implementation
│   │   └── evolutionary.py    # Evolutionary strategies
│   ├── communication/         # Communication-enabled agents
│   │   ├── emergent_comm.py   # Emergent communication
│   │   ├── attention_comm.py  # Attention-based communication
│   │   ├── market_based.py    # Market-based coordination
│   │   └── hierarchical.py    # Hierarchical coordination
│   └── meta_learning/         # Meta-learning agents
│       ├── maml.py            # Model-Agnostic Meta-Learning
│       ├── opponent_modeling.py # Opponent modeling
│       ├── population_training.py # Population-based training
│       └── continual_learning.py # Continual learning
├── environments/              # Multi-agent environments
│   ├── cooperative/           # Cooperative task environments
│   │   ├── resource_allocation.py # Resource allocation tasks
│   │   ├── team_navigation.py # Team navigation
│   │   ├── cooperative_games.py # Cooperative games
│   │   └── swarm_tasks.py      # Swarm robotics tasks
│   ├── competitive/           # Competitive environments
│   │   ├── adversarial_games.py # Adversarial games
│   │   ├── predator_prey.py   # Predator-prey scenarios
│   │   ├── auction_systems.py # Auction and bidding
│   │   └── competitive_games.py # Competitive games
│   ├── communication/         # Communication-required tasks
│   │   ├── emergent*comm*envs.py # Emergent communication tasks
│   │   ├── coordination_games.py # Coordination games
│   │   ├── signaling_games.py # Signaling and communication
│   │   └── multi*modal*comm.py # Multi-modal communication
│   └── real_world/            # Real-world applications
│       ├── autonomous_vehicles.py # Vehicle coordination
│       ├── smart_grid.py      # Grid management
│       ├── financial_trading.py # Trading systems
│       └── robotics_swarms.py # Swarm robotics
├── experiments/               # Experiment frameworks
│   ├── game_theory/           # Game theory experiments
│   │   ├── equilibrium_analysis.py # Equilibrium finding
│   │   ├── payoff_matrices.py # Payoff matrix analysis
│   │   ├── nash_equilibrium.py # Nash equilibrium computation
│   │   └── cooperative_games.py # Cooperative game analysis
│   ├── training_framework/    # Training orchestration
│   │   ├── multi*agent*trainer.py # Multi-agent training
│   │   ├── distributed_trainer.py # Distributed training
│   │   ├── evaluation_framework.py # Evaluation tools
│   │   └── hyperparameter_tuning.py # Parameter optimization
│   ├── communication/         # Communication experiments
│   │   ├── emergent*comm*exp.py # Emergent communication
│   │   ├── coordination_exp.py # Coordination experiments
│   │   ├── message_passing.py # Message passing analysis
│   │   └── communication_analysis.py # Communication analysis
│   ├── applications/          # Application-specific experiments
│   │   ├── autonomous_vehicles.py # Vehicle coordination
│   │   ├── smart_grid.py      # Grid management
│   │   ├── financial_trading.py # Trading experiments
│   │   └── robotics_swarms.py # Swarm experiments
│   └── analysis/              # Analysis and visualization
│       ├── performance_analysis.py # Performance metrics
│       ├── emergent_behavior.py # Emergent behavior analysis
│       ├── communication_analysis.py # Communication analysis
│       └── scalability_analysis.py # Scalability studies
├── utils/                     # General utilities
│   ├── setup.py               # Environment setup
│   ├── visualization.py       # Plotting and visualization
│   ├── data_collection.py     # Data collection tools
│   ├── evaluation.py          # Evaluation utilities
│   ├── communication.py       # Communication utilities
│   └── distributed_utils.py   # Distributed computing utilities
├── configs/                   # Configuration files
│   ├── agent_configs.py       # Agent configurations
│   ├── environment_configs.py # Environment settings
│   ├── training_configs.py    # Training parameters
│   └── experiment_configs.py  # Experiment settings
├── tests/                     # Unit tests
│   ├── test_agents.py         # Agent tests
│   ├── test_environments.py   # Environment tests
│   ├── test_communication.py  # Communication tests
│   └── test_distributed.py    # Distributed tests
├── requirements.txt           # Python dependencies
├── setup.py                   # Package setup
├── README.md                  # Project documentation
└── CA12.ipynb                 # This educational notebook
```

### Contents Overview

1. **Section 1**: Multi-Agent Foundations and Game Theory
2. **Section 2**: Cooperative Multi-Agent Learning
3. **Section 3**: Advanced Policy Gradient Methods
4. **Section 4**: Distributed Reinforcement Learning
5. **Section 5**: Communication and Coordination in Multi-Agent Systems
6. **Section 6**: Meta-Learning and Adaptation in Multi-Agent Systems
7. **Section 7**: Comprehensive Applications and Case Studies

In [ ]:
print("🤖 Multi-Agent Reinforcement Learning Environment Setup")
print(f"Device: {device}")
print(f"Available GPUs: {n_gpus}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

print("✅ Multi-Agent RL environment setup complete!")
print(
    f"🎯 Configuration: {ma_config.n_agents} agents, {ma_config.coordination_mechanism} coordination"
)
print("🚀 Ready for advanced multi-agent reinforcement learning!")


🤖 Multi-Agent Reinforcement Learning Environment Setup
Device: cpu
Available GPUs: 0
PyTorch version: 2.8.0
CUDA available: False
✅ Multi-Agent RL environment setup complete!
🎯 Configuration: 2 agents, centralized coordination
🚀 Ready for advanced multi-agent reinforcement learning!
🤖 Multi-Agent Reinforcement Learning Environment Setup
Device: cpu
Available GPUs: 0
PyTorch version: 2.8.0
CUDA available: False
✅ Multi-Agent RL environment setup complete!
🎯 Configuration: 2 agents, centralized coordination
🚀 Ready for advanced multi-agent reinforcement learning!


# Section 1: Multi-agent Foundations and Game Theory

## 1.1 Theoretical Foundation

### Multi-agent Reinforcement Learning (marl)

Multi-Agent Reinforcement Learning extends single-agent RL to environments with multiple learning agents. Key challenges include:

1. **Non-stationarity**: The environment appears non-stationary from each agent's perspective as other agents learn
2. **Partial observability**: Agents may have limited information about others' actions and observations
3. **Credit assignment**: Determining individual contributions to team rewards
4. **Scalability**: Computational complexity grows exponentially with number of agents
5. **Equilibrium concepts**: Finding stable solutions in multi-agent settings

### Game-theoretic Foundations

**Nash Equilibrium**: A strategy profile where no agent can improve by unilaterally changing strategy.

For agents $i = 1, ..., n$ with strategy spaces $S*i$ and utility functions $u*i(s*1, ..., s*n)$:
$$s^* = (s*1^*, ..., s*n^*) \text{ is a Nash equilibrium if } \forall i, s*i: u*i(s*i^*, s*{-i}^*) \geq u*i(s*i, s_{-i}^*)$$

**Pareto Optimality**: A strategy profile is Pareto optimal if no other profile improves at least one agent's utility without decreasing another's.

**Stackelberg Equilibrium**: Leader-follower game structure where one agent commits to a strategy first.

### Marl Paradigms

1. **Independent Learning**: Each agent treats others as part of the environment
2. **Joint Action Learning**: Agents learn about others' actions and adapt accordingly  
3. **Multi-Agent Actor-Critic (MAAC)**: Centralized training with decentralized execution
4. **Communication-Based Learning**: Agents exchange information to coordinate

### Cooperation Vs Competition Spectrum

- **Fully Cooperative**: Shared reward, common goal (e.g., team sports)
- **Fully Competitive**: Zero-sum game (e.g., adversarial settings)
- **Mixed-Motive**: Partially cooperative and competitive (e.g., resource sharing)

### Mathematical Formulation

**Multi-Agent MDP (MMDP)**:
- State space: $\mathcal{S}$
- Joint action space: $\mathcal{A} = \mathcal{A}*1 \times ... \times \mathcal{A}*n$
- Transition dynamics: $P(s'|s, a*1, ..., a*n)$
- Reward functions: $R*i(s, a*1, ..., a_n, s')$ for each agent $i$
- Discount factor: $\gamma \in [0, 1)$

**Policy Gradient in MARL**:
$$\nabla*{\theta*i} J*i(\theta*i) = \mathbb{E}*{\tau \sim \pi*{\theta}}[\sum*{t=0}^T \nabla*{\theta*i} \log \pi*{\theta*i}(a*{i,t}|o*{i,t}) A*i^t]$$

Where $A_i^t$ is agent $i$'s advantage at time $t$, which can be computed using various methods including multi-agent value functions.

---

In [ ]:
# Demonstrate game theory concepts and multi-agent environments
print("🎯 Game Theory Analysis Demo")
game_matrices = demonstrate_game_theory()
environments = test_multi_agent_env()


🎯 Game Theory Analysis Demo

1. Prisoner's Dilemma:
Player 1 payoff matrix:
[[-1 -3]
 [ 0 -2]]
Player 2 payoff matrix:
[[-1  0]
 [-3 -2]]
Nash equilibria: [(1, 1)]
Strategy (1, 1): Pareto optimal = False

2. Coordination Game:
Coordination game (both players have same payoffs):
[[2 0]
 [0 1]]
Nash equilibria: [(0, 0), (1, 1)]

🤖 Multi-Agent Environment Test
Testing cooperative environment:
Initial states shape: [(4,), (4,), (4,)]
Rewards (cooperative): [np.float64(-2.9035507487363637), np.float64(-2.9035507487363637), np.float64(-2.9035507487363637)]
All agents get same reward: True

Testing competitive environment:
Rewards (competitive): [np.float64(-2.1853851989532203), np.float64(-3.836553313738719), np.float64(-3.918044533204487)]
Agents get different rewards: True


# Section 2: Cooperative Multi-agent Learning

## 2.1 Centralized Training, Decentralized Execution (ctde)

The CTDE paradigm is fundamental to modern cooperative MARL:

**Training Phase**: 
- Central coordinator has access to global information
- Can compute joint value functions and coordinate policy updates
- Addresses non-stationarity through centralized critic

**Execution Phase**:
- Each agent acts based on local observations only
- No communication required during deployment
- Maintains scalability and robustness

### Multi-agent Actor-critic (maac)

**Centralized Critic**: Estimates joint action-value function $Q(s, a*1, ..., a*n)$

**Actor Update**: Each agent $i$ updates policy using centralized critic:
$$\nabla*{\theta*i} J*i = \mathbb{E}[\nabla*{\theta*i} \log \pi*{\theta*i}(a*i|o*i) \cdot Q^{\pi}(s, a*1, ..., a_n)]$$

**Critic Update**: Minimize joint TD error:
$$L(\phi) = \mathbb{E}[(Q*{\phi}(s, a*1, ..., a_n) - y)^2]$$
$$y = r + \gamma Q*{\phi'}(s', \pi*{\theta*1'}(o*1'), ..., \pi*{\theta*n'}(o_n'))$$

### Multi-agent Deep Deterministic Policy Gradient (maddpg)

Extension of DDPG to multi-agent settings:

1. **Centralized Critics**: Each agent maintains its own critic that uses global information
2. **Experience Replay**: Shared replay buffer with transitions $(s, a*1, ..., a*n, r*1, ..., r*n, s')$
3. **Target Networks**: Slow-updating target networks for stability

**Critic Loss for Agent $i$**:
$$L*i(\phi*i) = \mathbb{E}[(Q*{\phi*i}(s, a*1, ..., a*n) - y_i)^2]$$
$$y*i = r*i + \gamma Q*{\phi*i'}(s', \mu*{\theta*1'}(o*1'), ..., \mu*{\theta*n'}(o*n'))$$

**Actor Loss for Agent $i$**:
$$L*i(\theta*i) = -\mathbb{E}[Q*{\phi*i}(s, a*1|*{a*i=\mu*{\theta*i}(o*i)}, ..., a_n)]$$

### Counterfactual Multi-agent Policy Gradients (coma)

Uses counterfactual reasoning for credit assignment:

**Counterfactual Baseline**:
$$A*i(s, a) = Q(s, a) - \sum*{a*i'} \pi*i(a*i'|o*i) Q(s, a*{-i}, a*i')$$

This baseline removes the effect of agent $i$'s action, isolating its contribution to the team reward.

### Value Decomposition Networks (vdn)

Decomposes team value function into individual components:
$$Q*{tot}(s, a) = \sum*{i=1}^n Q*i(o*i, a_i)$$

**Advantages**:
- Individual value functions can be learned independently
- Naturally handles partial observability
- Maintains convergence guarantees under certain conditions

**Limitations**:
- Additivity assumption may be too restrictive
- Cannot represent complex coordination patterns

---

In [ ]:
# Demonstrate cooperative multi-agent learning algorithms
print("🤝 Cooperative Multi-Agent Learning Demo")

# Create MADDPG agents for demonstration
maddpg_agents = MADDPG(n_agents=2, obs_dim=4, action_dim=2)
print(f"Created MADDPG with {maddpg_agents.n_agents} agents")

# Create VDN agents for demonstration  
vdn_agents = VDN(n_agents=2, obs_dim=4, action_dim=2)
print(f"Created VDN with {vdn_agents.n_agents} agents")

print("✅ Cooperative learning algorithms ready!")



# Section 3: Advanced Policy Gradient Methods

## 3.1 Proximal Policy Optimization (ppo)

PPO addresses the challenge of step size in policy gradient methods through clipped objective functions.

### Ppo-clip Objective

**Probability Ratio**:
$$r*t(\theta) = \frac{\pi*\theta(a*t|s*t)}{\pi*{\theta*{old}}(a*t|s*t)}$$

**Clipped Objective**:
$$L^{CLIP}(\theta) = \hat{\mathbb{E}}*t[\min(r*t(\theta)A*t, \text{clip}(r*t(\theta), 1-\epsilon, 1+\epsilon)A_t)]$$

Where $\epsilon$ is the clipping parameter (typically 0.1-0.3) and $A_t$ is the advantage estimate.

### Trust Region Policy Optimization (trpo)

TRPO constrains policy updates to stay within a trust region:

**Objective**:
$$\max*\theta \hat{\mathbb{E}}*t[\frac{\pi*\theta(a*t|s*t)}{\pi*{\theta*{old}}(a*t|s*t)}A*t]$$

**Subject to**:
$$\hat{\mathbb{E}}*t[KL[\pi*{\theta*{old}}(\cdot|s*t), \pi*\theta(\cdot|s*t)]] \leq \delta$$

**Conjugate Gradient Solution**:
TRPO uses conjugate gradient to solve the constrained optimization problem:
$$g = \nabla*\theta L(\theta*{old})$$
$$H = \nabla*\theta^2 KL[\pi*{\theta*{old}}, \pi*\theta]$$
$$\theta*{new} = \theta*{old} + \sqrt{\frac{2\delta}{g^T H^{-1} g}} H^{-1} g$$

### Soft Actor-critic (sac)

SAC maximizes both expected return and entropy for better exploration:

**Objective**:
$$J(\pi) = \sum*{t=0}^T \mathbb{E}*{(s*t, a*t) \sim \rho*\pi}[r(s*t, a*t) + \alpha \mathcal{H}(\pi(\cdot|s*t))]$$

Where $\alpha$ is the temperature parameter controlling exploration-exploitation trade-off.

**Soft Q-Function Updates**:
$$J*Q(\phi) = \mathbb{E}*{(s*t, a*t, r*t, s*{t+1}) \sim \mathcal{D}}[\frac{1}{2}(Q*\phi(s*t, a*t) - y*t)^2]$$
$$y*t = r*t + \gamma \mathbb{E}*{a*{t+1} \sim \pi}[Q*{\phi'}(s*{t+1}, a*{t+1}) - \alpha \log \pi(a*{t+1}|s_{t+1})]$$

**Policy Updates**:
$$J*\pi(\theta) = \mathbb{E}*{s*t \sim \mathcal{D}, a*t \sim \pi*\theta}[\alpha \log \pi*\theta(a*t|s*t) - Q*\phi(s*t, a_t)]$$

### Advanced Advantage Estimation

**Generalized Advantage Estimation (GAE)**:
$$A*t^{GAE(\gamma, \lambda)} = \sum*{l=0}^\infty (\gamma\lambda)^l \delta_{t+l}^V$$

Where $\delta*t^V = r*t + \gamma V(s*{t+1}) - V(s*t)$ is the TD error.

GAE balances bias and variance:
- $\lambda = 0$: Low variance, high bias (TD error)
- $\lambda = 1$: High variance, low bias (Monte Carlo)

### Multi-agent Policy Gradient Extensions

**Multi-Agent PPO (MAPPO)**:
- Centralized value function: $V(s*1, ..., s*n)$
- Individual actor updates with shared value baseline
- Addresses non-stationarity through centralized training

**Multi-Agent SAC (MASAC)**:
- Individual entropy regularization per agent
- Shared experience replay buffer
- Independent policy and Q-function updates

---

In [ ]:
# Demonstrate advanced policy gradient methods
print("🎯 Advanced Policy Gradient Methods Demo")

# Create PPO agent for demonstration
ppo_agent = PPOAgent(obs_dim=4, action_dim=2, discrete=True)
print(f"Created PPO agent with obs_dim={4}, action_dim={2}")

# Create SAC agent for demonstration
sac_agent = SACAgent(obs_dim=4, action_dim=2)
print(f"Created SAC agent with obs_dim={4}, action_dim={2}")

# Create GAE buffer for demonstration
gae_buffer = GAEBuffer(size=1000, obs_dim=4, action_dim=2)
print(f"Created GAE buffer with size={1000}")

print("✅ Advanced policy gradient methods ready!")


# Section 4: Distributed Reinforcement Learning

## 4.1 Asynchronous Methods

Distributed RL enables parallel learning across multiple environments and workers, significantly improving sample efficiency and wall-clock training time.

### Asynchronous Advantage Actor-critic (a3c)

A3C runs multiple actor-learners in parallel, each interacting with a separate environment instance:

**Global Network Update**:
$$\theta*{global} \leftarrow \theta*{global} + \alpha \sum*{i=1}^{n*{workers}} \nabla \theta_i$$

**Local Gradient Accumulation**:
Each worker $i$ accumulates gradients over $t_{max}$ steps:
$$\nabla \theta*i = \sum*{t=1}^{t*{max}} \nabla \log \pi*{\theta*i}(a*t|s*t) A*t + \beta \nabla H(\pi*{\theta*i}(s_t))$$

Where $A_t$ is computed using n-step returns or GAE.

### Impala (importance Weighted Actor-learner Architecture)

IMPALA addresses the off-policy nature of distributed learning through importance sampling:

**V-trace Target**:
$$v*s = V(s*t) + \sum*{i=0}^{n-1} \gamma^i \prod*{j=0}^{i} c*{t+j} [r*{t+i} + \gamma V(s*{t+i+1}) - V(s*{t+i})]$$

**Importance Weights**:
$$\rho*t = \min(\bar{\rho}, \frac{\pi(a*t|s*t)}{\mu(a*t|s_t)})$$
$$c*t = \min(\bar{c}, \frac{\pi(a*t|s*t)}{\mu(a*t|s_t)})$$

Where $\mu` is the behavior policy and $\pi` is the target policy.

### Distributed Ppo (d-ppo)

Scales PPO to distributed settings while maintaining policy gradient guarantees:

1. **Rollout Collection**: Workers collect experience in parallel
2. **Gradient Aggregation**: Central server aggregates gradients
3. **Synchronized Updates**: Global policy update after each epoch

**Gradient Synchronization**:
$$g*{global} = \frac{1}{N} \sum*{i=1}^{N} g_i$$

Where $g_i$ is the gradient from worker $i$.

## 4.2 Evolutionary Strategies (es) in Rl

ES provides gradient-free optimization for RL policies:

**Population-Based Update**:
$$\theta*{t+1} = \theta*t + \alpha \frac{1}{\sigma \lambda} \sum*{i=1}^{\lambda} R*i \epsilon_i$$

Where:
- $\epsilon_i \sim \mathcal{N}(0, I)$ are random perturbations
- $R*i$ is the return achieved by perturbed policy $\theta*t + \sigma \epsilon_i$
- $\lambda$ is the population size

### Advantages of Es:
1. **Parallelizable**: Each worker evaluates different policy perturbation
2. **Gradient-free**: Works with non-differentiable rewards
3. **Robust**: Less sensitive to hyperparameters
4. **Communication efficient**: Only needs to share scalars (returns)

## 4.3 Multi-agent Distributed Learning

### Centralized Training Distributed Execution (ctde) at Scale

**Hierarchical Coordination**:
- **Global Coordinator**: Manages high-level strategy
- **Local Coordinators**: Handle subgroup coordination
- **Individual Agents**: Execute local policies

**Communication Patterns**:
1. **Broadcast**: Central coordinator broadcasts information to all agents
2. **Reduce**: Agents send information to central coordinator
3. **All-reduce**: All agents receive aggregated information from all others
4. **Ring**: Information flows in a circular pattern

### Parameter Server Architecture

**Parameter Server**: Maintains global model parameters
**Workers**: Pull parameters, compute gradients, push updates

**Asynchronous Updates**:
$$\theta*{t+1} = \theta*t - \alpha \sum*{i \in \text{available}} \nabla*i$$

**Advantages**:
- Fault tolerance through redundancy
- Scalable to thousands of workers
- Flexible resource allocation

---

In [ ]:
from agents.distributed_rl import (
    ParameterServer,
    A3CWorker,
    IMPALALearner,
    DistributedPPOCoordinator,
    EvolutionaryStrategy,
)
from agents.distributed_rl import (
    demonstrate_parameter_server,
    demonstrate_evolutionary_strategy,
)


# Demonstrate distributed reinforcement learning
print("🌐 Distributed Reinforcement Learning Demo")

# Demonstrate parameter server
param_server_demo = demonstrate_parameter_server()

# Demonstrate evolutionary strategy
es_demo = demonstrate_evolutionary_strategy()

print("✅ Distributed RL implementations ready!")



🖥️  Parameter Server Demo
Initial version: 0
After update: {'version': 1, 'updates': 1}

🧬 Evolutionary Strategy Demo
Generated population of size: 10
Parameter dimensionality: 58
✅ ES update completed


# Section 5: Communication and Coordination in Multi-agent Systems

## 5.1 Communication Protocols

Multi-agent systems often require sophisticated communication mechanisms to achieve coordination and share information effectively. This section explores various communication paradigms and their implementation in reinforcement learning contexts.

### Communication Types:
1. **Direct Communication**: Explicit message passing between agents
2. **Emergent Communication**: Learned communication protocols through RL
3. **Indirect Communication**: Environment-mediated information sharing
4. **Broadcast vs. Targeted**: Communication scope and recipients

### Mathematical Framework:
For agent $i$ sending message $m_i^t$ at time $t$:
$$m*i^t = \text{CommPolicy}*i(s*i^t, h*i^t)$$

Where $h_i^t$ is the communication history and the message influences other agents:
$$\pi*j(a*j^t | s*j^t, \{m*k^t\}_{k \neq j})$$

### Key Challenges:
- **Communication Overhead**: Balancing information sharing with computational cost
- **Partial Observability**: Deciding what information to communicate
- **Communication Noise**: Handling unreliable communication channels
- **Scalability**: Maintaining efficiency as the number of agents increases

## 5.2 Coordination Mechanisms

### Centralized Coordination:
- Global coordinator makes joint decisions
- Optimal but not scalable
- Single point of failure

### Decentralized Coordination:
- Agents coordinate through local interactions
- Scalable and robust
- May lead to suboptimal solutions

### Hierarchical Coordination:
- Multi-level coordination structure
- Combines benefits of centralized and decentralized approaches
- Natural for many real-world scenarios

### Market-based Coordination:
- Agents bid for tasks or resources
- Economically motivated coordination
- Natural load balancing

In [ ]:
from experiments.communication import (
    CommunicationChannel,
    AttentionCommunication,
    CoordinationMechanism,
)
from experiments.communication import MarketBasedCoordination, HierarchicalCoordination
from experiments.communication import (
    demonstrate_communication,
    demonstrate_coordination,
    demonstrate_emergent_communication,
)


# Demonstrate communication and coordination mechanisms
print("📡 Communication and Coordination Demo")

# Demonstrate communication mechanisms
comm_demo = demonstrate_communication()

# Demonstrate coordination mechanisms
coord_demo = demonstrate_coordination()

# Demonstrate emergent communication
emergent_demo = demonstrate_emergent_communication()

print("✅ Communication and coordination implementations ready!")



📡 Communication Mechanisms Demo
Message sent from agent 0 to agents [1, 2, 3]
Message shape: torch.Size([8])
Agent 1 received 1 messages

🤝 Coordination Mechanisms Demo
Market-based coordination result:
Task assignments: tensor([2, 3, 3])
Total value: 34.89

Hierarchical coordination levels: 2
Global decision shape: torch.Size([6])

🗣️  Emergent Communication Demo
Generated message: 13, log prob: -2.397
Action probabilities shape: torch.Size([4])
Value estimate: -0.017


# Section 6: Meta-learning and Adaptation in Multi-agent Systems

## 6.1 Meta-learning Foundations

Meta-learning, or "learning to learn," is particularly important in multi-agent systems where agents must quickly adapt to:
- New opponent strategies
- Changing team compositions  
- Novel task distributions
- Dynamic environment conditions

### Mathematical Framework:
Given a distribution of tasks $\mathcal{T}$, meta-learning aims to find parameters $\theta$ such that:
$$\theta^* = \arg\min*\theta \mathbb{E}*{\tau \sim \mathcal{T}} \left[ \mathcal{L}*\tau(\theta - \alpha \nabla*\theta \mathcal{L}_\tau(\theta)) \right]$$

Where $\alpha` is the inner learning rate and $\mathcal{L}_\tau` is the loss on task $\tau$.

## 6.2 Model-agnostic Meta-learning (maml) for Multi-agent Systems

MAML can be extended to multi-agent settings where agents must quickly adapt their policies to new scenarios:

### Multi-agent Maml Objective:
$$\min*{\theta*1, ..., \theta*n} \sum*{i=1}^n \mathbb{E}*{\tau \sim \mathcal{T}} \left[ \mathcal{L}*{\tau,i}(\phi_{i,\tau}) \right]$$

Where $\phi*{i,\tau} = \theta*i - \alpha*i \nabla*{\theta*i} \mathcal{L}*{\tau,i}(\theta_i)$

## 6.3 Few-shot Learning in Multi-agent Contexts

### Key Challenges:
1. **Opponent Modeling**: Quickly learning opponent behavior patterns
2. **Team Formation**: Adapting to new team compositions
3. **Strategy Transfer**: Applying learned strategies to new scenarios
4. **Communication Adaptation**: Adjusting communication protocols

### Applications:
- **Multi-Agent Navigation**: Adapting to new environments with different agents
- **Competitive Games**: Quickly learning counter-strategies
- **Cooperative Tasks**: Forming effective teams with unknown agents

## 6.4 Continual Learning in Dynamic Multi-agent Environments

### Catastrophic Forgetting Problem:
In multi-agent systems, agents may forget how to handle previously encountered opponents or scenarios when learning new ones.

### Solutions:
1. **Elastic Weight Consolidation (EWC)**: Protect important parameters
2. **Progressive Networks**: Expand capacity for new tasks
3. **Memory-Augmented Networks**: Store and replay important experiences
4. **Meta-Learning**: Learn how to quickly adapt without forgetting

## 6.5 Self-play and Population-based Training

### Self-play Evolution:
Agents improve by playing against previous versions of themselves or a diverse population of strategies.

### Population Diversity:
$$\text{Diversity} = \mathbb{E}*{\pi*i, \pi*j \sim P} [D(\pi*i, \pi_j)]$$

Where $P$ is the population and $D$ measures strategic distance between policies.

### Benefits:
- Robust strategy development
- Automatic curriculum generation
- Exploration of diverse play styles
- Prevention of exploitation vulnerabilities

In [ ]:
from agents.meta_learning import (
    MAMLAgent,
    OpponentModel,
    PopulationBasedTraining,
    SelfPlayTraining,
)
from agents.meta_learning import (
    demonstrate_maml,
    demonstrate_opponent_modeling,
    demonstrate_population_training,
)

print("🎓 Meta-Learning and Adaptation Systems")
# Demonstrate meta-learning and adaptation systems
print("🎓 Meta-Learning and Adaptation Systems")

# Demonstrate MAML
maml_demo = demonstrate_maml()

# Demonstrate opponent modeling
opponent_demo = demonstrate_opponent_modeling()

# Demonstrate population-based training
population_demo = demonstrate_population_training()

print("\n🚀 Meta-learning and adaptation implementations ready!")
print("✅ MAML, opponent modeling, and population-based training implemented!")

print("\n🚀 Meta-learning and adaptation implementations ready!")
print("✅ MAML, opponent modeling, and population-based training implemented!")


ImportError: cannot import name 'demonstrate_maml' from 'meta_learning' (/Users/tahamajs/Documents/uni/DRL/CAs/Solutions/CA12/meta_learning.py)

# Section 7: Comprehensive Applications and Case Studies

## 7.1 Multi-agent Resource Allocation

Resource allocation is a fundamental problem in multi-agent systems where agents must efficiently distribute limited resources while considering individual objectives and system-wide constraints.

### Problem Formulation:
- **Agents**: $\mathcal{A} = \{1, 2, ..., n\}$
- **Resources**: $\mathcal{R} = \{r*1, r*2, ..., r*m\}$ with quantities $\{q*1, q*2, ..., q*m\}$
- **Allocations**: $x_{i,j}$ = amount of resource $j$ allocated to agent $i$
- **Constraints**: $\sum*{i=1}^n x*{i,j} \leq q_j$ for all $j$

### Objective Functions:
1. **Utilitarian**: $\max \sum*{i=1}^n U*i(x_i)$
2. **Egalitarian**: $\max \min*i U*i(x_i)$
3. **Nash Social Welfare**: $\max \prod*{i=1}^n U*i(x_i)$

## 7.2 Autonomous Vehicle Coordination

Multi-agent reinforcement learning applications in autonomous vehicle systems present unique challenges in safety, efficiency, and scalability.

### Key Components:
- **Vehicle Agents**: Each vehicle as an independent learning agent
- **Communication**: V2V (Vehicle-to-Vehicle) and V2I (Vehicle-to-Infrastructure)
- **Objectives**: Safety, traffic flow optimization, fuel efficiency
- **Constraints**: Traffic rules, physical limitations, safety margins

### Coordination Challenges:
1. **Intersection Management**: Distributed traffic light control
2. **Highway Merging**: Cooperative lane changing and merging
3. **Platooning**: Formation and maintenance of vehicle platoons
4. **Emergency Response**: Coordinated response to accidents or hazards

## 7.3 Smart Grid Management

The smart grid represents a complex multi-agent system where various entities must coordinate for efficient energy distribution and consumption.

### Agent Types:
- **Producers**: Power plants, renewable energy sources
- **Consumers**: Residential, commercial, industrial users
- **Storage**: Battery systems, pumped hydro storage
- **Grid Operators**: Transmission and distribution system operators

### Challenges:
- **Demand Response**: Dynamic pricing and consumption adjustment
- **Load Balancing**: Real-time supply-demand matching
- **Renewable Integration**: Managing intermittent energy sources
- **Market Mechanisms**: Automated bidding and trading

## 7.4 Robotics Swarm Coordination

Swarm robotics involves coordinating large numbers of simple robots to achieve complex collective behaviors.

### Applications:
- **Search and Rescue**: Coordinated search patterns
- **Environmental Monitoring**: Distributed sensor networks
- **Construction**: Collaborative building and assembly
- **Military/Defense**: Autonomous drone swarms

### Technical Challenges:
- **Scalability**: Algorithms that work with hundreds or thousands of agents
- **Fault Tolerance**: Graceful degradation when agents fail
- **Communication Limits**: Bandwidth and range constraints
- **Real-time Coordination**: Fast decision making in dynamic environments

## 7.5 Financial Trading Systems

Multi-agent systems in financial markets involve multiple trading agents with different strategies and objectives.

### Agent Categories:
- **Market Makers**: Provide liquidity
- **Arbitrageurs**: Exploit price differences
- **Trend Followers**: Follow market momentum
- **Mean Reversion**: Bet on price corrections

### Market Dynamics:
- **Price Discovery**: Collective determination of asset values
- **Liquidity Provision**: Ensuring tradeable markets
- **Risk Management**: Controlling exposure and volatility
- **Regulatory Compliance**: Following trading rules and regulations

## 7.6 Game-theoretic Analysis Framework

### Nash Equilibrium in Multi-agent Rl:
For policies $\pi = (\pi*1, ..., \pi*n)$, a Nash equilibrium satisfies:
$$J*i(\pi*i^*, \pi*{-i}^*) \geq J*i(\pi*i, \pi*{-i}^*) \quad \forall \pi_i, \forall i$$

### Stackelberg Games:
Leader-follower dynamics where one agent commits to a strategy first:
$$\max*{\pi*L} J*L(\pi*L, \pi*F^*(\pi*L))$$
$$\text{s.t. } \pi*F^*(\pi*L) = \arg\max*{\pi*F} J*F(\pi*L, \pi_F)$$

### Cooperative Game Theory:
- **Shapley Value**: Fair allocation of cooperative gains
- **Core**: Stable coalition structures
- **Nucleolus**: Solution concept for transferable utility games

In [ ]:
from experiments.applications import (
    ResourceAllocationEnvironment,
    AutonomousVehicleEnvironment,
    SmartGridEnvironment,
    MultiAgentGameTheoryAnalyzer,
)
from experiments.applications import (
    demonstrate_resource_allocation,
    demonstrate_autonomous_vehicles,
    demonstrate_smart_grid,
)


print("🌟 Comprehensive Multi-Agent Applications")
resource_env = demonstrate_resource_allocation()
vehicle_env = demonstrate_autonomous_vehicles()
grid_env = demonstrate_smart_grid()

print("\n🚀 All comprehensive applications implemented!")
print("✅ Resource allocation, autonomous vehicles, and smart grid systems ready!")


🌍 Comprehensive Multi-Agent Applications
📊 Resource Allocation Demo
Resource allocation completed:
Rewards: tensor([43.2101, 23.6037, 62.6172])
Total allocation: 20
Resource Allocation State:
Current allocation:
tensor([[1, 1, 3, 1, 0],
        [1, 2, 0, 0, 1],
        [3, 2, 2, 0, 3]], dtype=torch.int32)
Remaining capacity: tensor([0, 0, 0, 4, 1])
Total utilization: 80.0%

🚗 Autonomous Vehicle Coordination Demo
Vehicle coordination step:
Average reward: -0.336
Collisions detected: False
Autonomous Vehicle Coordination:
Vehicle 0: Pos=11.6, Speed=11.6, Lane=1
Vehicle 1: Pos=27.7, Speed=1.0, Lane=0
Vehicle 2: Pos=64.9, Speed=11.5, Lane=0
Vehicle 3: Pos=92.1, Speed=12.1, Lane=2

⚡ Smart Grid Management Demo
Smart grid step:
Average reward: -4.887
Generation: 205.8, Demand: 152.5
Smart Grid - Time Step 1:
Total Demand: 164.1
Renewable Available: 14.3
Peak Hours: False

🎮 Game Theory Analysis Demo
Analyzing prisoners_dilemma:
Payoff matrix shape: torch.Size([2, 2, 2])
Nash Equilibria: [(1,